In [67]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [68]:
filename = "ABox.ttl"

In [69]:
text1 = '''CQ_3.1
Return all the risks with type "Cumulative process", the asset they are assiged to, the layers they exist within, and the documents that document them.
'''

query1 = '''
PREFIX abox: <http://purl.org/sirius/ontology/development/03/data/>
PREFIX tbox: <http://purl.org/sirius/ontology/development/03/schema/>

SELECT ?risk ?asset ?layer ?document
WHERE {
  ?risk_assessment a tbox:RiskAssessment ;
                  tbox:assignsRisk ?risk ;
                  tbox:assignsRiskTo ?asset .
  ?risk tbox:withType tbox:cumulative ;
        tbox:withinLayer ?layer ;
        tbox:isDocumentedBy ?document .
}
'''

In [70]:
text2 = '''CQ_3.2
Return all the risks existing with the layers "Region" or "Site", the asset they are assiged to, their type, and the start and end dates of the time intervals they exist in.
'''

query2 = '''
PREFIX abox: <http://purl.org/sirius/ontology/development/03/data/>
PREFIX tbox: <http://purl.org/sirius/ontology/development/03/schema/>

SELECT ?risk ?asset ?type ?time_interval_start ?time_interval_end
WHERE {
    ?risk_assessment a tbox:RiskAssessment ;
                    tbox:assignsRisk ?risk ;
                    tbox:assignsRiskTo ?asset .
    ?risk tbox:withType ?type ;
          tbox:withinLayer ?layer ;
          tbox:atTime ?time_interval .
    ?time_interval tbox:hasStartDate ?time_interval_start ;
                  tbox:hasEndDate ?time_interval_end . 
    FILTER ( ?layer = tbox:region || ?layer = tbox:site )
}
'''

In [72]:
queries = [(text1, query1),
           (text2, query2)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_3.1
Return all the risks with type "Cumulative process", the asset they are assiged to, the layers they exist within, and the documents that document them.

+-------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+
| risk                                                        | asset                                                          | layer                                                          | document                                                       |
|-------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------+----------------------------------------------------------------|
| http://purl.org/sirius/ontology/development/03/data/risk-01 |